In [1]:
from pliers.extractors import FarnebackOpticalFlowExtractor
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from pliers.stimuli import VideoStim

ModuleNotFoundError: No module named 'pliers'

In [2]:
#method to find the index of nearest value in an array
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx,array[idx]

In [3]:
stimulus = '../sourcedata/data/HBN/stimuli/hbn_stimuli/TP.mp4'
outdir = '../sourcedata/data/HBN/features/'
stim = os.path.splitext( os.path.split(stimulus)[1] )[0] # split stim name eg 'TP'

In [2]:
vs = VideoStim(stimulus)
ext = FarnebackOpticalFlowExtractor()
results = ext.transform(vs)

In [8]:
optic_flow = results.to_df(object_id='auto')

In [9]:
optic_flow.to_pickle(f"{outdir}{stim}_optic_flow.pkl")

In [10]:
#plt.plot(TP_optic_flow.total_flow)

In [34]:
flow = np.array(optic_flow.total_flow.tolist())
onset = np.array(optic_flow.onset.tolist())

In [54]:
flow_10hz = np.zeros(2032)

In [55]:
# downsample from 24hz to 10hz with 1d max pooling essentially. We take the max because we don't want to miss a spike/peak
previous_ind=0
for i,f in enumerate(flow_10hz):
    t=i/10
    nearest_ind,nearest_val = find_nearest(onset, t)
    flow_bin = np.append( flow[previous_ind:nearest_ind]  , 0 )
    flow_10hz[i] = max(flow_bin)
    previous_ind = nearest_ind

In [11]:
#plt.plot(flow_10hz)

In [58]:
np.save(f"{outdir}{stim}_optic_flow_10hz.npy",flow_10hz)

In [4]:
#flow_10hz = np.load('../sourcedata/data/HBN/features/TP_optic_flow_10hz.npy')

In [7]:
#export it as a tsv to visualize
onset_tsv = np.arange(0,203.2,0.1)
duration_tsv = np.ones(2032)-0.9
d = {"onset": onset_tsv, "duration": duration_tsv, "value": flow_10hz}
df = pd.DataFrame(d)
df.to_csv(f"{outdir}{stim}_optic_flow_10hz.tsv", sep='\t', index=False)